# Gaussian_Mixture_Model

Concepts used:

- `jit`: make our code move blazingly fast. See [exe_02_jit](../../exercises/exe_02_jit.ipynb) and [Jax jit docs](https://jax.readthedocs.io/en/latest/_autosummary/jax.jit.html)
- `vmap`: automatically vectorize our `e_step` and `m_step` across the matrices of data. See [exe_04_vmap](../../exercises/exe_04_vmap.ipynb) and [jax vmap docs](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html)
- composing `jit` and `vmap`. The functional programming nature of `jax` means that we can compose these higher-order functions, making our code look very clean. See the `e_step` for more. 

**Additional Concepts**
We also introduce the `RNG` and `PMAP` concepts at the very bottom, where we start our GMM from multiple random locations. For more information on the 

Before each newly introduced concept we briefly discuss the arguments and why the code is laid out the way it is :) 

In [ ]:
import jax.numpy as jnp
import jax
from jax.scipy.stats.norm import pdf as n_pdf
import numpy as np  
np.random.seed(123)

from sklearn.datasets import make_blobs

# Reporting Functions

Here we do not use a `vmap` and instead only use a `jit`. Note how we do: `to_exp = -0.5 * jnp.sum(diff @ inv * diff, axis=1)`, which looks nothing like the formula we see on Wikipedia

![](./gaussian_pdf.png)

Later on in the code we show an alternative form, where we use a `vmap` to avoid this strange looking code


In [ ]:
# We use a jit to speed up the function. a `jit` + `vmap` should achieve the same performance
@jax.jit
def gaussian_pdf(coor: jnp.array, mu_k: jnp.array, sigma_k: jnp.array) -> jnp.array:
    # PDF formula from: https://en.wikipedia.org/wiki/Multivariate_normal_distribution
    k = len(mu_k)
    t1 = (2 * jnp.pi) ** (-k / 2)
    t2 = jnp.linalg.det(sigma_k) ** (-0.5)

    inv = jnp.linalg.inv(sigma_k)
    diff = coor - mu_k
    to_exp = -0.5 * jnp.sum(diff @ inv * diff, axis=1)

    to_ret = t1 * t2 * jnp.exp(to_exp)

    assert len(to_ret) == len(coor)
    return to_ret

def log_likelihood(data, mu, sigma, pi, K):
    log_likelihood = 0
    for data_point in data:
        mixture_likelihood = 0
        for k in range(K):
            v = pi[k] * gaussian_pdf(
                jnp.expand_dims(data_point, axis=0), mu_k=mu[k], sigma_k=sigma[k]
            )
            mixture_likelihood += v
        log_likelihood += np.log(mixture_likelihood)

    return log_likelihood


# 1) Helper Functions for `e_step` 

- `_loglikelihood_gaussian` this is a log-likelihood calculation that operates on a single row of x, hence the $x_i$ notation. Because we have formulated it this way, we see that our code from before (after applying the log) has become `0.5 * diff @ sigma_inv @ diff`

- `ll_gaussian` this applies the `_loglikelihood_gaussian` across elements of the first argument. Notice how we have `in_axes=(0, None, None, None)`, which means we vectorize over the 0-th axis of the first argument, which is the data matrix in our example.

- `calculate_normalizer`: no special tricks here, just calculating the normalizer for the [log-sum-exp](https://en.wikipedia.org/wiki/LogSumExp) trick

In [ ]:

@jax.jit
def _loglikelihood_gaussian(x_i: jnp.array, cls_prior_k: jnp.array, mu_k: jnp.array, sigma_k: jnp.array) -> jnp.array:
    """
    Calculate the LL for a single point
    Args:
        x_i: vector of shape (1, num_feats)
        mu_k: vector of shape (1, num_feats)
        sigma_k: matrix of shape (num_feats, num_feats)

    """
    k = len(mu_k)
    sigma_inv = jnp.linalg.inv(sigma_k)
    sigma_det = jnp.linalg.det(sigma_k)
    log_det_sigma = jnp.log(sigma_det)

    diff = x_i - mu_k
    t1 = -0.5 * k * jnp.log(2 * jnp.pi)
    t2 = -0.5 * log_det_sigma
    t3 = -0.5 * diff @ sigma_inv @ diff

    return t1 + t2 + t3 + jnp.log(cls_prior_k)


@jax.jit
def ll_gaussian(X, cls_prior_k, mu_k, sigma_k):
    return jax.vmap(
        fun=_loglikelihood_gaussian,
        in_axes=(0, None, None, None)
    )(X, cls_prior_k, mu_k, sigma_k)

@jax.jit
def calculate_normalizer(log_prob_arr: jnp.ndarray) -> jnp.ndarray:
    """
    For the log-sum-exp
    Args:
        log_prob_arr:

    Returns:

    """
    _max = jnp.max(log_prob_arr, axis=0)
    return _max + jnp.log(jnp.sum(
        jnp.exp(log_prob_arr - _max),
        axis=0
    ))


# 2) EM Algorithm

Reference: [Wiki: EM_algorithm_and_GMM_model](https://en.wikipedia.org/wiki/EM_algorithm_and_GMM_model)

In the **E-step** we calculate the weights for the updates based on how likely a point is to have come from one of the specified clusters

In the **M-step** we calculate the update for the parameters



## `e_step`

we use the `ll_gaussian` which maps `_loglikelihood_gaussian` over each row of the dataset. We have actually `vmap`-ed the `ll_gaussian` 
over each of the parameters of the `mus`, `sigmas` and `cls_priors`, all of which are of shape (4, ...), which means that we are "indexing" over all of them at the same time


In [ ]:

@jax.jit
def e_step(X, mus, sigmas, cls_priors):
    ll_gaussian_over_parameters = jax.vmap(
        ll_gaussian,
        in_axes=(None, 0, 0, 0)
    )
    _responsibilities = ll_gaussian_over_parameters(X, cls_priors, mus, sigmas)
    normalizer = calculate_normalizer(_responsibilities)
    return jnp.exp(_responsibilities - normalizer).T[0]

## m-step

`_m_step_single`: we calculate the mu and sigma for a single row, which we will sum up later

`_m_step`: given a specific class, k, get the corresponding `responsibility` and `mu`, then calculate the updated `mu`, `sigma`, and class prior. Our `X` if of shape (N, M) and our `resp_k` is of shape (N, 1), so we map over the 0-th axis

`m_step`: we map our `_m_step`, that is conditioned on the class parameters, across the different class parameters. The `responsibilities` are of shape (N, K) and our `mus` are of shape (K, D), so when we do the `vmap`, we map 
over the 0-th axis of `mu` and the 1st axis of `responsibilities`

In [ ]:

@jax.jit
def _m_step_single(x_i, mu_k, resp_nk):
    """
    Calculate the individual values for mu and sigma
    """

    mu_new = x_i * resp_nk
    diff = x_i - mu_k
    sigma_new = resp_nk * jnp.outer(diff, diff)
    return mu_new, sigma_new

@jax.jit
def _m_step(X, mu_k, resp_k):
    N_k = jnp.sum(resp_k)
    to_ave_mus, to_ave_sigmas = jax.vmap(
        _m_step_single,
        in_axes=(0, None, 0)
    )(X, mu_k, resp_k)

    mus = jnp.sum(to_ave_mus, axis=0) / N_k
    sigmas = jnp.sum(to_ave_sigmas, axis=0) / N_k
    cls_prior = N_k / len(X)
    return mus, sigmas, cls_prior

@jax.jit
def m_step(X, mus, responsibilities):
    mus, sigmas, cls_prior = jax.vmap(
        _m_step,
        in_axes=(None, 0, 1)
    )(X, mus, responsibilities)

    return mus, sigmas, jnp.expand_dims(cls_prior, -1)

# Putting it all together

In [ ]:
def EM_GMM(
        data: np.ndarray,
        guess_num_classes,
        # Initial guesses
        mus, sigmas, cls_probs,
        
        verbose=False
        
):
    counter = 0
    ll_container = []
    TOL = 0.00001
    ll_container.append(np.inf)

    while True:  # Run until converges
        # e-step
        responsibilities = e_step(data, mus, sigmas, cls_probs)

        # m-step
        mus, sigmas, cls_probs = m_step(data, mus, responsibilities)
        # Recalculate the log-likelihood
        ll_curr = float(log_likelihood(data, mus, sigmas, cls_probs, guess_num_classes))

        if np.abs(ll_container[-1] - ll_curr) < TOL:
            print(f"Converged to within {TOL} after: {counter} iterations")
            break

        ll_container.append(float(ll_curr))
        if verbose:
            print(f"Data Log-Likelihood at iteration: {counter} = {ll_curr:.6f}")
        counter += 1

    responsibilities = e_step(data, mus, sigmas, cls_probs)
    return mus, sigmas, cls_probs.T, responsibilities.T, ll_container[1:]
    # -------------------------- #



# Problem Setup

Our problem formulation is where there are 4 actual clusters (we do not know this). In a real-world scenario, we do not know the number of clusters ahead-of-time. 

In [ ]:
unknown_centers = np.asarray([
    [1, -1],  # bottom left
    [5, 5],  # middle
    [8, 7],  # mid-right
    [10, 0]  # bottom right
])

def make_ds(centers):
    points_in_classes = [30, 50, 20, 5]
    ################################################
    # Initial Guesses
    ################################################
    # Randomly increase/ decrease by 10% each way
    scale = (np.random.randint(low=0, high=20, size=centers.shape) - 10) / 100

    initial_mu_guesses = centers + (centers * scale)
    return make_blobs(points_in_classes, centers=centers), initial_mu_guesses

(X, y), mus = make_ds(unknown_centers)

N, M = X.shape
K = len(np.unique(y))

# Modeling

In [ ]:
sigmas = np.asarray([np.cov(X.T) for _ in range(K)])
cls_probs = np.asarray([1 / K for _ in range(K)]).T
cls_probs = np.expand_dims(cls_probs, axis=-1)
mus, sigmas, cls_priors, _, lls = EM_GMM(
    X,
    mus=mus,
    sigmas=sigmas,
    cls_probs=cls_probs,
    guess_num_classes=K,
    verbose=True
)

# Result Investigation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(lls)

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(mu, sigma, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Modified based on function from: https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The Axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    pearson = sigma[0, 1]/np.sqrt(sigma[0, 0] * sigma[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(sigma[0, 0]) * n_std
    # calculating the standard deviation of y ...
    scale_y = np.sqrt(sigma[1, 1]) * n_std

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mu[0], mu[1])

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(15, 10))

colors = ["r", "g", "b", "y"]

for i, c in enumerate(colors):
    
    # Plot the centers
    plt.scatter(unknown_centers[i, 0], unknown_centers[i, 1], c=c, marker="o", label=f"Cluster: {i} True Center")
    plt.scatter(mus[i, 0], mus[i, 1], c=c, marker="^", label=f"Cluster: {i} Inferred Center")
    
    print(f"True vs Inferred: {unknown_centers[i]} and {mus[i]}")

    # Plot the standard deviations
    mask = y == i
    masked_points = X[mask]
    mu_x = np.mean(masked_points, axis=0)
    sigma = np.cov(masked_points[:, 0], masked_points[:, 1])
    confidence_ellipse(mu_x, sigma,  ax=axs, n_std=1, edgecolor=c, linestyle="-")
    confidence_ellipse(mu_x, sigma, ax=axs, n_std=2, edgecolor=c, linestyle="-")
    confidence_ellipse(mu_x, sigma, ax=axs, n_std=3, edgecolor=c, linestyle="-")


    confidence_ellipse(mus[i], sigmas[i],  ax=axs, n_std=1, edgecolor=c, linestyle="--")
    confidence_ellipse(mus[i], sigmas[i], ax=axs, n_std=2, edgecolor=c, linestyle="--")
    confidence_ellipse(mus[i], sigmas[i], ax=axs, n_std=3, edgecolor=c, linestyle="--")
plt.legend(loc="best")
